In [2]:
import cv2
import numpy as np
from skimage.filters import frangi, sobel
from matplotlib import pyplot as plt

# Część na 3.0

In [3]:
def show_image(img, title="test"):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Wczytanie zdjęcia

In [4]:
#blada gałka oczna na czarnym tle
#image = cv2.imread('zdjecia/2_good.JPG') # 11.42 / 10.99

#bardziej czerwona głka oczna na czrnym tle
#image = cv2.imread('zdjecia/8_good.JPG') # 21.93 / 21.25

#duża pomarańczowa gałka oczna na czarnym tle
image = cv2.imread('zdjecia/Image_03R.jpg') # 51.76 / 56.1

image = cv2.resize(image, (800, 800))

gray_to_find_circle = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

## Wstępne przetworzenie obrazu

In [5]:
def thresh_show(image, th1):
    
    image2 = cv2.threshold(image, th1, 255, cv2.THRESH_BINARY)[1]
    #show_image(image2)
    
    return cv2.findContours(image2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [6]:
#rozmycie

gray_to_find_circle = cv2.GaussianBlur(gray_to_find_circle, (5, 5), 0) # medianblur, guassianblur ?
show_image(image, title="blur")
image = cv2.GaussianBlur(image, (5, 5), 0) # medianblur, guassianblur ?

#wyostrzenie
#obraz potrzebny do odnalezienia gałki ocznej
kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
gray_to_find_circle = cv2.filter2D(gray_to_find_circle, -1, kernel)
show_image(gray_to_find_circle, title="filter")

#obraz na którym będziemy dalej pracować
image = cv2.filter2D(image, -1, kernel)
image = cv2.filter2D(image, -1, kernel)
show_image(image, title="filter")

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#image = frangi(image, black_ridges=False)
#show_image(image, title="frangi")

In [7]:
# szukamy gałki ocznej
contours, hierarchy = thresh_show(gray_to_find_circle, 0)

contours_approved = []
(cx, cy) , radius = (0, 0), 0
global approxx
maxArea = 0
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, .03 * cv2.arcLength(cnt, True), True)
    if len(approx) >= 7:
        area = cv2.contourArea(cnt)
        (cx, cy), radius = cv2.minEnclosingCircle(cnt)
        circleArea = radius * radius * np.pi
        if maxArea<area:
            contours_approved = []
            maxArea = area
            contours_approved.append(cnt)
            approxx = approx

# wiodący obraz z konturami gałki ocznej i wypełnieniem
cv2.drawContours(image, contours_approved, -1, (255, 0, 0), 3) 
show_image(image, title="wiodacy obraz z konturami galki ocznej")

In [8]:
rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

fill_color = [255, 255, 255]
mask_value = 255 

stencil  = np.zeros(rgb.shape[:-1]).astype(np.uint8)
cv2.fillPoly(stencil, contours_approved, mask_value)

sel      = stencil != mask_value # select everything that is not mask_value
rgb[sel] = fill_color            # and fill it with fill_color

show_image(rgb, title="wiodacy obraz z konturami galki ocznej i wypelnieniem" )

## Właściwe przetworzenie obrazu

In [9]:
def nothing(x):
    pass

def trackbar_thresholds(image):
    cv2.namedWindow('myImage')

    cv2.createTrackbar('threshold1', 'myImage', 0, 255, nothing)
    cv2.createTrackbar('threshold2', 'myImage', 0, 255, nothing)

    
    image2 = image

    while(1):
        cv2.imshow('myImage', image2)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
        # get current positions of four trackbars
        th1 = cv2.getTrackbarPos('threshold1', 'myImage')
        th2 = cv2.getTrackbarPos('threshold2', 'myImage')
        # canny
        image2 = cv2.Canny(image, th1, th2)
        
    cv2.destroyAllWindows()

In [10]:
image = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

In [23]:
#trackbar_thresholds(image)

#image = cv2.Canny(image, 100, 230)
# wykrywanie krawędzi
#image = frangi(image, black_ridges=True)
image_mean = np.mean(image)
print(image_mean)
wspol = 1.3 * image_mean # 0 - 15

# thresholding
ret,image2 = cv2.threshold(image,wspol, 255, cv2.THRESH_BINARY)
#show_image(image2)



137.8359359375


## Sprawdzenie

In [24]:
odp = cv2.imread('zdjecia/Image_03R_1stHO.png')
odp = cv2.resize(odp, (800, 800))

#show_image(odp)
i = 0
max_val = 640000
 
for odp_, res_ in zip(np.nditer(odp), np.nditer(image2)):
    if odp_ == res_:
        i += 1
    
print(i/max_val)


0.627815625


# Część na 5.0

38